In [2]:
!pip install voxelmorph
!pip install nilearn
!pip install nibabel

In [10]:
import os
import subprocess
import multiprocessing as mp
from nilearn import plotting
import nibabel as nib
import voxelmorph as vxm
import tensorflow as tf
from tensorflow.keras import layers
from voxelmorph.layers import VxmDense

root_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

def main():
    # Change the working directory to the "data" folder
    os.chdir('../data')

    # Get the list of patient folders
    patient_folders = [folder for folder in os.listdir() if os.path.isdir(folder) and folder.startswith('E')]

    # Define the input image filenames and output names
    input_output_filenames = {
        'T1w_1mm.nii.gz': 'T1w_align.nii.gz',
        'T2w_1mm_noalign.nii.gz': 'T2w_align.nii.gz',
        'ADC_deformed.nii.gz': 'ADC_align.nii.gz',
        'FA_deformed.nii.gz': 'FA_align.nii.gz'
    }

    # Load the T1w reference image
    ref_image_path = os.path.join('input', patient_folders[0], 'T1w_1mm.nii.gz')
    ref_image = nib.load(ref_image_path)
    
    # Load the pretrained VoxelMorph model called synthmorph
    model_path = os.path.join(root_dir, 'code','models', 'brains-dice-vel-0.5-res-16-256f.h5')
    # Register custom VoxelMorph layers
    custom_objects = {'VxmDense': VxmDense}
    model = tf.keras.models.load_model(model_path, compile=False)

    # Perform registration on each patient folder for each input image and save the output with the new name
    for patient_folder in patient_folders:
        for input_image_filename, output_image_filename in input_output_filenames.items():
            input_image_path = os.path.join('input', patient_folder, input_image_filename)
            output_image_path = os.path.join(patient_folder, output_image_filename)

            # Load the input image
            moving_image = nib.load(input_image_path)

            # Perform registration
            warped_image, warp_field = model.register(ref_image, moving_image)

            # Save registered image
            nib.save(warped_image, output_image_path)

if __name__ == "__main__":
    main()


AttributeError: module 'voxelmorph.tf.layers' has no attribute 'VxmDense'